# **Importing Packages**

In [101]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
!pip install scikit-plot
import scikitplot as skplt

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from statsmodels.discrete.discrete_model import Logit
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score,recall_score,confusion_matrix,classification_report,roc_auc_score,roc_curve, precision_score, f1_score
import sklearn as sk
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus


from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from decimal import Decimal

from google.colab import files

# **Data Preprocessing & Feature Engineering**

In [102]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pd.read_csv('/content/drive/My Drive/data_modeling_10s_MIX.csv')

In [0]:
data = data.drop(["Unnamed: 0","id"], axis=1)

# removing the columns containing information on s1 and s3 sensors 
data = data[data.columns.drop(list(data.filter(regex ='s3')))]  
data = data[data.columns.drop(list(data.filter(regex ='s1')))]

In [105]:
# Here we need to combine the two columns
data["Target"].replace({"Mixed_walking_with_little_pauses": "Mixed walking"}, inplace=True)
data["Target"].replace({"Mixed_walking_includes_stairs_up": "Mixed walking"}, inplace=True)
data["Target"].value_counts()

Mixed walking    663
Walking          629
Sitting          257
Standing         231
Jogging          195
Climbing down    132
Climbing up      120
Stairs up        105
Stairs down       64
No activity       12
Name: Target, dtype: int64

In [106]:
data["Kg"] = np.where(data["Kg"] <=75, 0, 1)
data["Kg"].value_counts()

0    1938
1     470
Name: Kg, dtype: int64

In [0]:
data["x_std_div_z_std"] = data["x_std"] / data["z_std"]
data["y_std_div_z_std"] = data["y_std"] / data["z_std"]

data["x_tvar_div_z_tvar"] = data["x_tvar"] / data["z_tvar"]
data["y_tvar_div_z_tvar"] = data["y_tvar"] / data["z_tvar"]
 
data["s2_peak_len_50_110_20_300_diff"] = abs(data["s2_peak_len_20_300"] - data["s2_peak_len_50_110"])
data["x_peak_prom_stand_diff_y"] = abs(data["x_peak_prom_stand"] - data["y_peak_prom_stand"])
data["x_peak_prom_sit_diff_y"] = abs(data["x_peak_prom_sit"] - data["y_peak_prom_sit"])
data["x_peak_len_diff_y"] = abs(data["x_peak_len"] - data["y_peak_len"])
data["s2_peak_prom_stand_sit_diff"] = abs(data["s2_peak_prom_stand"] - data["s2_peak_prom_sit"])
data["x_peak_len_diff_y_peak_len_50_200"] = abs(data["x_peak_len_50_200"] - data["y_peak_len_50_200"]) 
data["x_y_peak_len_20_150_diff_50_110"] = abs(data["x_peak_len_20_150"] - data["y_peak_len_50_110"])
data["s2_peak_len_50_110_20_300_diff"] = abs(data["s2_peak_len_20_300"] - data["s2_peak_len_50_110"])

data["z_peak_len_sq"] = np.square(data["z_peak_len"])
data["y_peak_len_sq"] = np.square(data["y_peak_len"])
data["y_z_peak_len_mul_squares"] = data["z_peak_len_sq"] * data ["y_peak_len_sq"]

data["s2_prom_div_height"] = (data["s2_peak_prom"] / data["s2_peak_height"]).fillna(0)
data["s2_prom_div_height"] = data["s2_prom_div_height"].replace([np.inf, -np.inf], 0)
data["x_prom_div_height"] = (data["x_peak_prom"] / data["x_peak_height"]).fillna(0)
data["x_prom_div_height"] = data["x_prom_div_height"].replace([np.inf, -np.inf], 0)
data["y_prom_div_height"] = (data["y_peak_prom"] / data["y_peak_height"]).fillna(0)
data["y_prom_div_height"] = data["y_prom_div_height"].replace([np.inf, -np.inf], 0)
data["z_prom_div_height"] = (data["z_peak_prom"] / data["z_peak_height"]).fillna(0)
data["z_prom_div_height"] = data["z_prom_div_height"].replace([np.inf, -np.inf], 0)


data["s2_entropy"] = np.where(data["s2_entropy"]==0, 0, 1)
data["x_entropy"] = np.where(data["x_entropy"]==0, 0, 1)
data["y_entropy"] = np.where(data["y_entropy"]==0, 0, 1)
data["z_entropy"] = np.where(data["z_entropy"]==0, 0, 1)


data["s2_kstatvar"] = np.sqrt(data["s2_kstatvar"])
data["x_kstatvar"] = np.sqrt(data["x_kstatvar"])
data["y_kstatvar"] = np.sqrt(data["y_kstatvar"])
data["z_kstatvar"] = np.sqrt(data["z_kstatvar"]) 

data["s2_peak_height_std"] = np.nanstd(np.where(np.isclose(data["s2_peak_height"],0), np.nan, data["s2_peak_height"]))
data["s2_variation_peak_height"] = (data["s2_peak_height_std"] / data["s2_peak_height"]).fillna(0)
data["s2_variation_peak_height"] = data["s2_variation_peak_height"].replace([np.inf, -np.inf], 0)
data["x_peak_height_std"] = np.nanstd(np.where(np.isclose(data["x_peak_height"],0), np.nan, data["x_peak_height"]))
data["x_variation_peak_height"] = (data["x_peak_height_std"] / data["x_peak_height"]).fillna(0)
data["x_variation_peak_height"] = data["x_variation_peak_height"].replace([np.inf, -np.inf], 0)
data["y_peak_height_std"] = np.nanstd(np.where(np.isclose(data["y_peak_height"],0), np.nan, data["y_peak_height"]))
data["y_variation_peak_height"] = (data["y_peak_height_std"] / data["y_peak_height"]).fillna(0)
data["y_variation_peak_height"] = data["y_variation_peak_height"].replace([np.inf, -np.inf], 0)
data["z_peak_height_std"] = np.nanstd(np.where(np.isclose(data["z_peak_height"],0), np.nan, data["z_peak_height"]))
data["z_variation_peak_height"] = (data["z_peak_height_std"] / data["z_peak_height"]).fillna(0)
data["z_variation_peak_height"] = data["z_variation_peak_height"].replace([np.inf, -np.inf], 0)

In [0]:
data = data.drop(["s2_max","x_max", "y_max", "z_max", "s2_min", "x_min", "y_min", "z_min", "y_mean","z_mean", "x_mean", "s2_mean", "y_moment","z_moment", "x_moment", "s2_moment", 
                  "x_variation", "s2_variation", "y_variation","z_variation", "s2_sem", "x_sem", "y_sem", "z_sem", "x_std", "y_std", "z_std", "s2_peak_prom_sit", "x_peak_prom_sit", "y_peak_prom_sit",   
                  "s2_peak_prom_stand", "x_peak_prom_stand", "y_peak_prom_stand", "x_peak_len_50_200", "y_peak_len_50_200", "x_peak_len_20_150", "y_peak_len_50_110", "s2_peak_len_50_110", "s2_peak_len_20_300", 
                  "x_peak_len", "y_peak_len", "z_peak_len", "s2_peak_height", "s2_peak_prom", "x_peak_height", "x_peak_prom", "y_peak_height", "y_peak_prom", "z_peak_height", "z_peak_prom",
                  "x_tvar", "y_tvar", "z_tvar"], axis=1)

In [0]:
data = data[['s2_median', 'x_median','y_median', 'z_median', 's2_std',  's2_skew', 'x_skew', 'y_skew', 'z_skew','s2_kurtosis', 'x_kurtosis', 'y_kurtosis', 'z_kurtosis', 
             's2_kstatvar', 'x_kstatvar', 'y_kstatvar', 'z_kstatvar',  's2_peak_len', 's2_peak_width', 'x_peak_width', 'y_peak_width', 'z_peak_width',
             'x_y_corr', 'z_y_corr','x_z_corr', 's2_tvar', 's2_weight_pred_k', 'x_weight_pred_k', 'y_weight_pred_k', 'z_weight_pred_k', 'x_std_div_z_std', 'y_std_div_z_std', 
             'y_z_peak_len_mul_squares', 'x_peak_prom_stand_diff_y','x_peak_prom_sit_diff_y', 
             'x_peak_len_diff_y', 's2_peak_prom_stand_sit_diff','x_peak_len_diff_y_peak_len_50_200','x_y_peak_len_20_150_diff_50_110','s2_peak_len_50_110_20_300_diff',
             's2_entropy', 'x_entropy', 'y_entropy','z_entropy', 's2_prom_div_height', 'x_prom_div_height', 'y_prom_div_height', 'z_prom_div_height', 
             's2_variation_peak_height', 'x_variation_peak_height', 'y_variation_peak_height', 'z_variation_peak_height', 'x_tvar_div_z_tvar', 'y_tvar_div_z_tvar', 'Kg', 'Target', 'Name']]  

In [110]:
dict = {'Walking' :  0,  'Mixed walking' :1, 'Sitting':2, 'Standing':3, 'Jogging':4, 'Climbing down':5,                          
          'Climbing up':6,  'Stairs up':7,  'Stairs down':8, 'Mixed walking  with little pauses':9, 'Mixed walking  includes stairs up':10, 'No activity':11}

data["Target"] = data.Target.map(dict)
data["Target"].value_counts()

1     663
0     629
2     257
3     231
4     195
5     132
6     120
7     105
8      64
11     12
Name: Target, dtype: int64

In [0]:
data = data.drop(["x_peak_prom_sit_diff_y"], axis=1)

In [0]:
y = data["Target"]
x = data.drop(["Target", "Name"], axis=1)

# **Train Test split**

In [0]:
unique_name=data['Name'].unique()
np.random.seed(42)
random_names_train = np.random.choice(a = unique_name, size = 6, replace = False)  
train = pd.DataFrame()
for name in random_names_train:
  pre_train = data[data['Name'] == name] 
  train = train.append(pre_train)


y_train=train["Target"]
x_train=train.drop(["Target", "Name"],axis=1)


random_names_test = []
for letter in unique_name:
  if letter not in random_names_train:
    random_names_test.append(letter)

test = pd.DataFrame()
for names in random_names_test:
  pre_test = data[data['Name'] == names] 
  test = test.append(pre_test)


y_test=test["Target"]                       
x_test=test.drop(["Target", "Name"], axis=1)

# **Scaling**

In [0]:
scaler = RobustScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)

In [0]:
scaler = RobustScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)

In [0]:
x_train.columns = [['s2_median', 'x_median', 'y_median', 'z_median', 's2_std', 's2_skew', 'x_skew', 'y_skew', 'z_skew', 's2_kurtosis', 'x_kurtosis', 'y_kurtosis', 'z_kurtosis', 
              's2_kstatvar', 'x_kstatvar', 'y_kstatvar', 'z_kstatvar', 's2_peak_len', 's2_peak_width', 'x_peak_width', 'y_peak_width', 'z_peak_width', 'x_y_corr', 'z_y_corr', 'x_z_corr',
              's2_tvar', 's2_weight_pred_k', 'x_weight_pred_k', 'y_weight_pred_k', 'z_weight_pred_k', 'x_std_div_z_std', 'y_std_div_z_std', 'y_z_peak_len_mul_squares',
              'x_peak_prom_stand_diff_y', 'x_peak_len_diff_y', 's2_peak_prom_stand_sit_diff', 'x_peak_len_diff_y_peak_len_50_200', 'x_y_peak_len_20_150_diff_50_110', 
              's2_peak_len_50_110_20_300_diff', 's2_entropy', 'x_entropy', 'y_entropy', 'z_entropy', 's2_prom_div_height', 'x_prom_div_height', 'y_prom_div_height', 
              'z_prom_div_height', 's2_variation_peak_height', 'x_variation_peak_height', 'y_variation_peak_height', 'z_variation_peak_height', 'x_tvar_div_z_tvar', 'y_tvar_div_z_tvar', 'Kg']]

x_test.columns = [['s2_median', 'x_median', 'y_median', 'z_median', 's2_std', 's2_skew', 'x_skew', 'y_skew', 'z_skew', 's2_kurtosis', 'x_kurtosis', 'y_kurtosis', 'z_kurtosis', 
              's2_kstatvar', 'x_kstatvar', 'y_kstatvar', 'z_kstatvar', 's2_peak_len', 's2_peak_width', 'x_peak_width', 'y_peak_width', 'z_peak_width', 'x_y_corr', 'z_y_corr', 'x_z_corr',
              's2_tvar', 's2_weight_pred_k', 'x_weight_pred_k', 'y_weight_pred_k', 'z_weight_pred_k', 'x_std_div_z_std', 'y_std_div_z_std', 'y_z_peak_len_mul_squares',
              'x_peak_prom_stand_diff_y', 'x_peak_len_diff_y', 's2_peak_prom_stand_sit_diff', 'x_peak_len_diff_y_peak_len_50_200', 'x_y_peak_len_20_150_diff_50_110', 
              's2_peak_len_50_110_20_300_diff', 's2_entropy', 'x_entropy', 'y_entropy', 'z_entropy', 's2_prom_div_height', 'x_prom_div_height', 'y_prom_div_height', 
              'z_prom_div_height', 's2_variation_peak_height', 'x_variation_peak_height', 'y_variation_peak_height', 'z_variation_peak_height', 'x_tvar_div_z_tvar', 'y_tvar_div_z_tvar', 'Kg']]

In [0]:
y_test_index = y_test.index
x_test.index = y_test_index

y_train_index = y_train.index
x_train.index = y_train_index

# **Models**

## **Logistic Regression**

In [130]:
logit_m = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(x_train, y_train)

logit_m_train = logit_m.predict(x_train)
logit_m_test = logit_m.predict(x_test)

logit_train_score = logit_m.score(x_train,y_train)*100
logit_test_score = logit_m.score(x_test,y_test)*100

print("Train:",logit_tran_score)
print("Test:", logit_test_score)

cr_train_log = classification_report(y_train, logit_m_train)
cr_test_log = classification_report(y_test, logit_m_test) 

print("Train:", cr_train_log)
print("Test:", cr_test_log)

Train: 87.01015965166908
Test: 51.262135922330096
Train:               precision    recall  f1-score   support

           0       0.85      0.90      0.87       348
           1       0.84      0.81      0.83       380
           2       0.83      0.89      0.86       139
           3       0.81      0.78      0.79       121
           4       1.00      1.00      1.00       120
           5       0.90      0.87      0.88        90
           6       0.91      0.86      0.88        80
           7       0.96      0.89      0.92        62
           8       1.00      1.00      1.00        38

    accuracy                           0.87      1378
   macro avg       0.90      0.89      0.89      1378
weighted avg       0.87      0.87      0.87      1378

Test:               precision    recall  f1-score   support

           0       0.48      0.68      0.56       281
           1       0.43      0.40      0.42       283
           2       0.55      0.35      0.43       118
           3   

In [131]:
param_logit_m={'class_weight':[None,'balanced'],
            'C':np.linspace(0.0001,10,400),
            'penalty': ['l1','l2', "elasticnet"]}

gs_logit_m = GridSearchCV(estimator=logit_m,
                        param_grid=param_logit_m,
                        cv=5,verbose=1,n_jobs=-1).fit(x_train, y_train)

Fitting 5 folds for each of 2400 candidates, totalling 12000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:    9.7s
[Parallel(n_jobs=-1)]: Done 1464 tasks      | elapsed:   54.6s
[Parallel(n_jobs=-1)]: Done 3464 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 6264 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 9864 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done 12000 out of 12000 | elapsed:  7.5min finished


In [0]:
gs_logit_m.best_params_

{'C': 4.862206766917293, 'class_weight': None, 'penalty': 'l2'}

In [132]:
logit2_m = LogisticRegression(C= 0.8, penalty="l2", class_weight=None, random_state=0, solver='lbfgs', multi_class='multinomial').fit(x_train, y_train)

logit_m_train2 = logit2_m.predict(x_train)
logit_m_test2 = logit2_m.predict(x_test)

logit_train_score2 = logit2_m.score(x_train,y_train)*100
logit_test_score2 = logit2_m.score(x_test,y_test)*100

print("Train for Tuned Logit:",logit_train_score2)
print("Test for Tuned Logit:", logit_test_score2)

cr_train_log2 = classification_report(y_train, logit_m_train2)
cr_test_log2 = classification_report(y_test, logit_m_test2) 

print("Train:", cr_train_log2)
print("Test:", cr_test_log2)

Train for Tuned Logit: 86.5021770682148
Test for Tuned Logit: 51.165048543689316
Train:               precision    recall  f1-score   support

           0       0.84      0.91      0.87       348
           1       0.84      0.79      0.81       380
           2       0.83      0.89      0.86       139
           3       0.80      0.78      0.79       121
           4       0.99      1.00      1.00       120
           5       0.90      0.87      0.88        90
           6       0.88      0.86      0.87        80
           7       0.96      0.89      0.92        62
           8       1.00      1.00      1.00        38

    accuracy                           0.87      1378
   macro avg       0.89      0.89      0.89      1378
weighted avg       0.87      0.87      0.86      1378

Test:               precision    recall  f1-score   support

           0       0.48      0.69      0.56       281
           1       0.43      0.40      0.41       283
           2       0.56      0.35     

## **SVM: Before and after GridSeach**

In [127]:
from sklearn import svm

svm_m = svm.SVC(decision_function_shape="ovo").fit(x_train, y_train)

svm_m_train_pred = svm_m.predict(x_train)
svm_m_test_pred = svm_m.predict(x_test)

svm_train_score = svm_m.score(x_train, y_train)*100 
svm_test_score = svm_m.score(x_test, y_test)*100

print("Train for SVM:",svm_train_score)
print("Test for SVM:",svm_test_score)

cr_train_svm = classification_report(y_train, svm_m_train_pred)
cr_test_svm = classification_report(y_test, svm_m_test_pred) 

print("Train:", cr_train_svm)
print("Test:", cr_test_svm)

Train for SVM: 71.91582002902757
Test for SVM: 52.33009708737863
Train:               precision    recall  f1-score   support

           0       0.62      0.92      0.74       348
           1       0.69      0.74      0.71       380
           2       0.79      0.85      0.82       139
           3       0.78      0.74      0.76       121
           4       0.99      0.97      0.98       120
           5       0.92      0.13      0.23        90
           6       1.00      0.07      0.14        80
           7       0.92      0.58      0.71        62
           8       1.00      0.37      0.54        38

    accuracy                           0.72      1378
   macro avg       0.86      0.60      0.63      1378
weighted avg       0.77      0.72      0.69      1378

Test:               precision    recall  f1-score   support

           0       0.51      0.73      0.60       281
           1       0.40      0.47      0.43       283
           2       0.73      0.53      0.61       118


In [128]:
svm_grid_params = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

gs_svm = GridSearchCV(estimator=svm_m,
                  param_grid=svm_grid_params, cv=3, n_jobs=-1, verbose=1).fit(x_train,y_train)

gs_svm.best_params_

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  1.1min finished


{'C': 10, 'kernel': 'linear'}

In [129]:
from sklearn import svm

svm_m2 = svm.SVC(C=0.8, kernel='linear', decision_function_shape="ovo").fit(x_train, y_train)

svm_m_train_pred2 = svm_m2.predict(x_train)
svm_m_test_pred2 = svm_m2.predict(x_test)

svm_train_score2 = svm_m2.score(x_train, y_train)*100 
svm_test_score2 = svm_m2.score(x_test, y_test)*100

print("Train for SVM Tuned SVM:",svm_train_score2)
print("Test for SVM Tuned SVM:",svm_test_score2)

cr_train_svm2 = classification_report(y_train, svm_m_train_pred2)
cr_test_svm2 = classification_report(y_test, svm_m_test_pred2) 

print("Train:", cr_train_svm2)
print("Test:", cr_test_svm2)

Train for SVM Tuned SVM: 90.34833091436865
Test for SVM Tuned SVM: 52.71844660194175
Train:               precision    recall  f1-score   support

           0       0.88      0.92      0.90       348
           1       0.86      0.86      0.86       380
           2       0.90      0.92      0.91       139
           3       0.91      0.82      0.86       121
           4       1.00      1.00      1.00       120
           5       0.94      0.91      0.93        90
           6       0.95      0.93      0.94        80
           7       1.00      0.94      0.97        62
           8       1.00      1.00      1.00        38

    accuracy                           0.90      1378
   macro avg       0.94      0.92      0.93      1378
weighted avg       0.90      0.90      0.90      1378

Test:               precision    recall  f1-score   support

           0       0.49      0.67      0.56       281
           1       0.41      0.42      0.42       283
           2       0.68      0.42 

## **Random Forest: Before and after GridSearch**

In [118]:
rf_m = RandomForestClassifier(n_estimators=1000, criterion="entropy", max_depth=10, random_state=42).fit(x_train, y_train)

rf_m_train_pred = rf_m.predict(x_train)
rf_m_test_pred = rf_m.predict(x_test)

rf_train_score = rf_m.score(x_train, y_train)*100
rf_test_score = rf_m.score(x_test, y_test)*100

print("Train for Random Forest:", rf_train_score)
print("Test for Random Forest:", rf_test_score)

cr_train_rf = classification_report(y_train, rf_m_train_pred)
cr_test_rf = classification_report(y_test, rf_m_test_pred)

print("Train:", cr_train_rf)
print("Test:", cr_test_rf)

# Return the mean accuracy on the given test data and labels.

Train for Random Forest: 99.63715529753266
Test for Random Forest: 54.757281553398066
Train:               precision    recall  f1-score   support

           0       0.99      1.00      0.99       348
           1       1.00      0.99      0.99       380
           2       1.00      1.00      1.00       139
           3       1.00      1.00      1.00       121
           4       1.00      1.00      1.00       120
           5       0.99      1.00      0.99        90
           6       1.00      1.00      1.00        80
           7       1.00      1.00      1.00        62
           8       1.00      1.00      1.00        38

    accuracy                           1.00      1378
   macro avg       1.00      1.00      1.00      1378
weighted avg       1.00      1.00      1.00      1378

Test:               precision    recall  f1-score   support

           0       0.54      0.70      0.61       281
           1       0.40      0.48      0.43       283
           2       0.76      0.47

In [119]:
param_rf_m={"max_depth":range(1,20),
           "min_samples_leaf":range(10,300,10),
           "class_weight":["balanced",None],  
            "criterion":["entropy","gini"]      
             }

gs_rf_m = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                   param_grid=param_rf_m, cv=3, n_jobs=-1).fit(x_train,y_train)

gs_rf_m.best_params_

{'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 8,
 'min_samples_leaf': 10}

In [121]:
rf2_m = RandomForestClassifier(class_weight='balanced', n_estimators=1000, criterion = "entropy", min_samples_leaf=10, max_depth=8, random_state=42).fit(x_train, y_train)

rf_m_train_pred2 = rf2_m.predict(x_train)
rf_m_test_pred2 = rf2_m.predict(x_test)

rf_train_score2 = rf2_m.score(x_train, y_train)*100
rf_test_score2 = rf2_m.score(x_test, y_test)*100

print("Train for Tuned Random Forest:", rf_train_score2)
print("Test for Tuned Random Forest:", rf_test_score2)

cr_train_rf2 = classification_report(y_train, rf_m_train_pred2)
cr_test_rf2 = classification_report(y_test, rf_m_test_pred2)

print("Train:", cr_train_rf2)
print("Test:", cr_test_rf2)

Train for Tuned Random Forest: 87.73584905660378
Test for Tuned Random Forest: 55.92233009708738
Train:               precision    recall  f1-score   support

           0       0.88      0.97      0.92       348
           1       0.99      0.63      0.77       380
           2       0.92      0.95      0.94       139
           3       0.75      0.94      0.84       121
           4       0.98      1.00      0.99       120
           5       0.77      0.98      0.86        90
           6       0.83      0.99      0.90        80
           7       0.72      0.98      0.83        62
           8       0.97      1.00      0.99        38

    accuracy                           0.88      1378
   macro avg       0.87      0.94      0.89      1378
weighted avg       0.90      0.88      0.87      1378

Test:               precision    recall  f1-score   support

           0       0.52      0.75      0.62       281
           1       0.54      0.33      0.41       283
           2       0.7

In [122]:
feature_importances_rf = pd.DataFrame(rf2_m.feature_importances_,index = x_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances_rf

,importance
x_kstatvar,0.096918
s2_std,0.084188
s2_tvar,0.073973
x_skew,0.059907
s2_kstatvar,0.055739
y_kstatvar,0.054526
x_median,0.053677
z_kstatvar,0.048552
z_median,0.043547
x_kurtosis,0.039593


In [0]:
x_train_d = x_train.drop([ 'x_y_corr', 'y_std_div_z_std',	's2_median',	'y_tvar_div_z_tvar',	'z_y_corr', 'x_prom_div_height',	'x_variation_peak_height',	'z_kurtosis',	's2_peak_width',	'y_skew',	
                          's2_prom_div_height',	'z_entropy',	'y_weight_pred_k',	'x_z_corr', 'y_z_peak_len_mul_squares',	'x_peak_len_diff_y', 'x_weight_pred_k',	's2_weight_pred_k',	's2_peak_len',	
                          'z_prom_div_height',	'y_variation_peak_height',	'y_peak_width',	's2_variation_peak_height',	's2_peak_prom_stand_sit_diff','y_prom_div_height',	'x_peak_width',	
                          'x_peak_len_diff_y_peak_len_50_200',	'Kg',	'z_peak_width',	'z_variation_peak_height',	'z_weight_pred_k',	'y_entropy', 'x_entropy',	's2_entropy', 'x_y_peak_len_20_150_diff_50_110', 'x_weight_pred_k'], axis=1)
                         
x_test_d = x_test.drop([ 'x_y_corr', 'y_std_div_z_std',	's2_median',	'y_tvar_div_z_tvar',	'z_y_corr', 'x_prom_div_height',	'x_variation_peak_height',	'z_kurtosis',	's2_peak_width',	'y_skew',	
                          's2_prom_div_height',	'z_entropy',	'y_weight_pred_k',	'x_z_corr', 'y_z_peak_len_mul_squares',	'x_peak_len_diff_y', 'x_weight_pred_k',	's2_weight_pred_k',	's2_peak_len',	
                          'z_prom_div_height',	'y_variation_peak_height',	'y_peak_width',	's2_variation_peak_height',	's2_peak_prom_stand_sit_diff','y_prom_div_height',	'x_peak_width',	
                          'x_peak_len_diff_y_peak_len_50_200',	'Kg',	'z_peak_width',	'z_variation_peak_height',	'z_weight_pred_k',	'y_entropy', 'x_entropy',	's2_entropy', 'x_y_peak_len_20_150_diff_50_110', 'x_weight_pred_k'], axis=1)

# **KNN: Before and after GridSearch**

In [124]:
knn_m = KNeighborsClassifier(n_neighbors = 7).fit(x_train_d, y_train)

knn_m_train_pred = knn_m.predict(x_train_d)
knn_m_test_pred = knn_m.predict(x_test_d)

knn_train_score = knn_m.score(x_train_d, y_train)*100
knn_test_score = knn_m.score(x_test_d, y_test)*100

print("Train for KNN:", knn_train_score)
print("Test for KNN:", knn_test_score)

cr_train_knn = classification_report(y_train, knn_m_train_pred)
cr_test_knn = classification_report(y_test, knn_m_test_pred) 

print("Train:", cr_train_knn)
print("Test:", cr_test_knn)

Train for KNN: 82.22060957910014
Test for KNN: 54.1747572815534
Train:               precision    recall  f1-score   support

           0       0.83      0.94      0.88       348
           1       0.75      0.74      0.75       380
           2       0.79      0.80      0.79       139
           3       0.77      0.58      0.66       121
           4       1.00      0.98      0.99       120
           5       0.85      0.91      0.88        90
           6       0.87      0.84      0.85        80
           7       0.81      0.71      0.76        62
           8       0.97      0.84      0.90        38

    accuracy                           0.82      1378
   macro avg       0.85      0.82      0.83      1378
weighted avg       0.82      0.82      0.82      1378

Test:               precision    recall  f1-score   support

           0       0.53      0.66      0.59       281
           1       0.46      0.63      0.53       283
           2       0.63      0.44      0.52       118
 

In [125]:
knn_grid_params = {"n_neighbors":range(1,20),
          "weights":["uniform","distance"],
          "metric":["euclidean","manhattan"]        
            }
gs_knn = GridSearchCV(estimator=knn_m,
                  param_grid=knn_grid_params, cv=3, n_jobs=-1, verbose=1).fit(x_train,y_train)

gs_knn.best_params_

Fitting 3 folds for each of 76 candidates, totalling 228 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 228 out of 228 | elapsed:    9.6s finished


{'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'distance'}

In [126]:
knn_m2 = KNeighborsClassifier(n_neighbors = 11, metric="manhattan", weights="distance").fit(x_train_d, y_train)

knn_m_train_pred2 = knn_m2.predict(x_train_d)
knn_m_test_pred2 = knn_m2.predict(x_test_d)

knn_train_score2 = knn_m2.score(x_train_d, y_train)*100
knn_test_score2 = knn_m2.score(x_test_d, y_test)*100

print("Train for Tuned KNN:", knn_train_score2)
print("Test for Tuned KNN:", knn_test_score2)

cr_train_knn2 = classification_report(y_train, knn_m_train_pred2)
cr_test_knn2 = classification_report(y_test, knn_m_test_pred2) 

print("Train:", cr_train_knn2)
print("Test:", cr_test_knn2)

Train for Tuned KNN: 100.0
Test for Tuned KNN: 54.271844660194176
Train:               precision    recall  f1-score   support

           0       1.00      1.00      1.00       348
           1       1.00      1.00      1.00       380
           2       1.00      1.00      1.00       139
           3       1.00      1.00      1.00       121
           4       1.00      1.00      1.00       120
           5       1.00      1.00      1.00        90
           6       1.00      1.00      1.00        80
           7       1.00      1.00      1.00        62
           8       1.00      1.00      1.00        38

    accuracy                           1.00      1378
   macro avg       1.00      1.00      1.00      1378
weighted avg       1.00      1.00      1.00      1378

Test:               precision    recall  f1-score   support

           0       0.51      0.64      0.57       281
           1       0.47      0.56      0.51       283
           2       0.66      0.50      0.57       118